<a href="https://colab.research.google.com/github/Struth-Rourke/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/Assignment_Regression_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [54]:
df.head()
df.drop(['description', 'display_address', 'street_address'], axis = 1, inplace = True)
df.head()

,bathrooms,bedrooms,created,latitude,longitude,price,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,40.7145,-73.9425,3000,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,40.7947,-73.9667,5465,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,40.7388,-74.0018,2850,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,40.7539,-73.9677,3275,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,40.8241,-73.9493,3350,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [55]:
# Number of Total Rooms (Bathrooms + Bedrooms + Dining Room)
df['number_of_rooms'] = df['bathrooms'] + df['bedrooms'] + df['dining_room']

# In unit Amenities + Fitness: Hardwood Floors, Dishwasher, Laundry (in unit), Fitness Center
df['amenities'] = df['hardwood_floors'] + df['dishwasher'] + df['laundry_in_unit'] + df['fitness_center']

# Pet Friendly (Cats and/or Dogs)
df['pet_friendly'] = df['cats_allowed'] + df['dogs_allowed']

# Total Outdoor Space + Balcony or Terrace
df['total_outdoor_space'] = df['roof_deck'] + df['outdoor_space'] + df['balcony'] + df['terrace'] + df['garden_patio'] + df['common_outdoor_space']

df.head()

,bathrooms,bedrooms,created,latitude,longitude,price,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,number_of_rooms,amenities,pet_friendly,total_outdoor_space
0,1.5,3,2016-06-24 07:54:24,40.7145,-73.9425,3000,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,0,0,0
1,1.0,2,2016-06-12 12:19:27,40.7947,-73.9667,5465,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,1,2,0
2,1.0,1,2016-04-17 03:26:41,40.7388,-74.0018,2850,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2,0,0
3,1.0,1,2016-04-18 02:22:02,40.7539,-73.9677,3275,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,1,0,0
4,1.0,4,2016-04-28 01:32:41,40.8241,-73.9493,3350,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0,0,0,0


In [20]:
train = df[df['created'] < '2016-06']
print(train.shape)
train.head()

(31844, 35)


,bathrooms,bedrooms,created,latitude,longitude,price,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,number_of_rooms,amenities,pet_friendly,total_outdoor_space
2,1.0,1,2016-04-17 03:26:41,40.7388,-74.0018,2850,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2,0,0
3,1.0,1,2016-04-18 02:22:02,40.7539,-73.9677,3275,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,1,0,0
4,1.0,4,2016-04-28 01:32:41,40.8241,-73.9493,3350,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0,0,0,0
5,2.0,4,2016-04-19 04:24:47,40.7429,-74.0028,7995,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.0,0,0,0
6,1.0,2,2016-04-27 03:19:56,40.8012,-73.9660,3600,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,0,2,0


In [21]:
test = df[df['created'] >= '2016-06']
print(test.shape)
test.head()

(16973, 35)


,bathrooms,bedrooms,created,latitude,longitude,price,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,number_of_rooms,amenities,pet_friendly,total_outdoor_space
0,1.5,3,2016-06-24 07:54:24,40.7145,-73.9425,3000,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,0,0,0
1,1.0,2,2016-06-12 12:19:27,40.7947,-73.9667,5465,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,1,2,0
11,1.0,1,2016-06-03 03:21:22,40.8448,-73.9396,1675,low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0,0,0
14,1.0,1,2016-06-01 03:11:01,40.7584,-73.9648,3050,low,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2,0,0
24,2.0,4,2016-06-07 04:39:56,40.7391,-73.9936,7400,medium,1,1,1,1,1,1,0,0,1,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,6.0,3,2,3


In [75]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

### TRAINING DATA ###

# Instantiate the Linear Model
linearreg_model = LinearRegression()

# Features and Target
features = ['number_of_rooms', 'amenities']
target = ['price']

# X train / test paired with Features
X_train = train[features]

# y train / test paired with Target
y_train = train[target]

# Fitting the Model to the training data
linearreg_model.fit(X_train, y_train)

# Predicting the Price based on training data
y_pred_train = linearreg_model.predict(X_train)

# Mean Absolute Error
mse = mean_squared_error(y_train, y_pred_train)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)
print('Mean Absolute Error (Testing) =', mae)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

print('.....')

# Coefficients and Intercept
print('Intercept =', linearreg_model.intercept_)
print('Coefficients =', linearreg_model.coef_)


Mean Absolute Error (Testing) = 879.1258964129345
Mean Squared Error: 1725608.0429367886
Root Mean Squared Error: 1313.624011251617
Mean Absolute Error: 879.1258964129345
R^2: 0.44425366797190546
.....
Intercept = [1283.01175896]
Coefficients = [[726.0965152  173.59293649]]


In [76]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

### TESTING DATA ###

# Instantiate the Linear Model
linearreg_model = LinearRegression()

# Features and Target
features = ['number_of_rooms', 'amenities']
target = ['price']

# X test paired with Features
X_test = test[features]

# y test paired with Target
y_test = test[target]

# Fitting the Model to the Testing Data
linearreg_model.fit(X_test, y_test)

# Predicting the Price based on training data
y_pred_test = linearreg_model.predict(X_test)

# Mean Absolute Error
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)
print('Mean Absolute Error (Testing) =', mae)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

print('.....')

# Coefficients and Intercept
print('Intercept =', linearreg_model.intercept_)
print('Coefficients =', linearreg_model.coef_)

Mean Absolute Error (Testing) = 892.3375577411225
Mean Squared Error: 1724307.3217533554
Root Mean Squared Error: 1313.1288290770847
Mean Absolute Error: 892.3375577411225
R^2: 0.44520736084127954
.....
Intercept = [1278.48334657]
Coefficients = [[736.80761913 156.06405886]]


In [77]:
### TRAINING DATA -- NEW FEATURES ###

# Instantiate the Linear Model
linearreg_model = LinearRegression()

# Features and Target
features = ['pet_friendly', 'total_outdoor_space']
target = ['price']

# X train / test paired with Features
X_train = train[features]

# y train / test paired with Target
y_train = train[target]

# Fitting the Model to the training data
linearreg_model.fit(X_train, y_train)

# Predicting the Price based on training data
y_pred_train = linearreg_model.predict(X_train)

# Mean Absolute Error
mse = mean_squared_error(y_train, y_pred_train)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)
print('Mean Absolute Error (Testing) =', mae)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

print('.....')

# Coefficients and Intercept
print('Intercept =', linearreg_model.intercept_)
print('Coefficients =', linearreg_model.coef_)

Mean Absolute Error (Testing) = 1175.7735747819308
Mean Squared Error: 2991313.433817105
Root Mean Squared Error: 1729.5413940744827
Mean Absolute Error: 1175.7735747819308
R^2: 0.03662278604069025
.....
Intercept = [3343.86195853]
Coefficients = [[ 77.51190756 352.47452535]]


In [79]:
### TESTING DATA -- NEW FEATURES ###

# Instantiate the Linear Model
linearreg_model = LinearRegression()

# Features and Target
features = ['pet_friendly', 'total_outdoor_space']
target = ['price']

# X test paired with Features
X_test = test[features]

# y test paired with Target
y_test = test[target]

# Fitting the Model to the Testing Data
linearreg_model.fit(X_test, y_test)

# Predicting the Price based on training data
y_pred_test = linearreg_model.predict(X_test)

# Mean Absolute Error
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)
print('Mean Absolute Error (Testing) =', mae)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

print('.....')

# Coefficients and Intercept
print('Intercept =', linearreg_model.intercept_)
print('Coefficients =', linearreg_model.coef_)

Mean Absolute Error (Testing) = 1167.7612900523307
Mean Squared Error: 2956917.5407659207
Root Mean Squared Error: 1719.568998547578
Mean Absolute Error: 1167.7612900523307
R^2: 0.04861734012233587
.....
Intercept = [3326.67765847]
Coefficients = [[ 83.50274392 432.14831665]]
